## Setup

In [35]:
%pip install geopandas
%pip install geopy
%pip install gmaps
%pip install pyarrow
%pip install pandas-geojson
%pip install python-google-places
%pip install googlemaps
%pip install ipyleaflet
%pip install rauth

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Created wheel for rauth: filename=rauth-0.7.3-py3-none-any.whl size=16065 sha256=429cbeb117bfa28d71c9be7f144311bed87f00666058b5644ae18831f20d55c0
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/e8/bf/f8/fe5ecf3f58dd944bdf853ce2d995b3dddef2a2fea48d1f84bc
Successfully built rauth
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import datetime
import pandas as pd
import googlemaps
from pandas_geojson import to_geojson
from ipyleaflet import Map, GeoJSON, FullScreenControl, Marker, MarkerCluster, AwesomeIcon
from ipywidgets import Layout, HTML

cfg = json.load(open('secure/config.json'))
gmaps = googlemaps.Client(key=cfg['google_maps_api_key'])

## Load school list from CSV and add GPS coordinates

In [11]:
df = pd.read_csv('schulsuche_export.csv', delimiter=';', encoding='ISO-8859-1')
# Filter schools of interest
schools_of_interest = pd.concat([df[df.Schultyp=='Gymnasium'], df[df.Schultyp=='Freie Waldorfschule']], ignore_index=True)

In [12]:
def address_to_gps(strasse, plz, ort):
    address_joined = ', '.join([strasse, str(plz), ort])
    geocode_result = gmaps.geocode(address_joined)
    res = (None, None)
    if geocode_result:
        res = (geocode_result[0]['geometry']['location']['lat'],
               geocode_result[0]['geometry']['location']['lng'])
    # location = geolocator.geocode(address_joined)
    return res

schools_of_interest[['latitude', 'longitude']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: address_to_gps(row.Strasse, row.PLZ, row.Ort), axis=1
    ).tolist(), index=schools_of_interest.index)

In [13]:
# Write to file to preserve GPS coordinates
schools_of_interest.to_parquet('parquet/schools_gps.parquet', engine='pyarrow')

## Add travel time to Munich via public transport

In [14]:
def add_travel_info(lat, lng, destination):
    # To get a typical travel time, we set departure time to next Monday morning
    today = datetime.date.today()
    next_monday_0700 = datetime.datetime.combine(
        today + datetime.timedelta(days=-today.weekday(), weeks=1),
        datetime.datetime.strptime('07:00', '%H:%M').time()
    )
    
    directions_result = gmaps.directions(origin={'lat': lat, 'lng': lng},
                                         destination=destination,
                                         mode="transit",
                                         departure_time=next_monday_0700)
    res = (None, None)
    if directions_result and directions_result[0]['legs']:
        res = (directions_result[0]['legs'][0]['duration']['value'] // 60, # convert to minutes
               directions_result[0]['legs'][0]['distance']['value'] // 1000) # convert to kilometer
    return res      

schools_of_interest[['travel_duration_min', 'travel_distance_km']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: add_travel_info(row.latitude, row.longitude, 'Münchner Freiheit'), axis=1
    ).tolist(), index=schools_of_interest.index)

# Write to file to save added travel times.
schools_of_interest.to_parquet('parquet/schools_gps_travel.parquet', engine='pyarrow')

## Add markers and show schools on map

In [2]:
def add_school_map_marker(row):
    icon_type = {
        'Gymnasium': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='blue',
            icon_color='black',
            spin=False
        ),
        'Freie Waldorfschule': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='green',
            icon_color='black',
            spin=False
        )
    }
    icon_no_public_transport = AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='black',
            icon_color='white',
            spin=False
        )
    marker_icon = icon_type[row.Schultyp] if not(pd.isna(row.travel_duration_min)) else icon_no_public_transport
    marker = Marker(icon=marker_icon, location=(row.latitude, row.longitude), draggable=False, title=row.Name)
    message = HTML()
    message.value = f'<a href="https://{row.Homepage}">{row.Name}</a><p>travel time: {row.travel_duration_min} min</p>'
    marker.popup = message
    return marker

# Load previously saved data
schools_of_interest = pd.read_parquet('parquet/schools_gps_travel.parquet', engine="pyarrow")

# Add map markers
schools_of_interest[['marker']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: add_school_map_marker(row), axis=1
    ).tolist(), index=schools_of_interest.index)

munich = (48.144227, 11.560515)
m = Map(center=munich, zoom=9, layout=Layout(width='100%', height='900px'))
m.add_control(FullScreenControl())

waldorf_cluster = MarkerCluster(
    markers=(schools_of_interest[schools_of_interest.Schultyp=='Freie Waldorfschule']['marker'].tolist())
)
m.add_layer(waldorf_cluster);
gymnasium_cluster = MarkerCluster(
    markers=(schools_of_interest[schools_of_interest.Schultyp=='Gymnasium']['marker'].tolist())
)
m.add_layer(gymnasium_cluster);

display(m)

Map(center=[48.144227, 11.560515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…